In [2]:
import pandas as pd
import openai
from dotenv import load_dotenv
import json
import os

In [3]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
#load data
df = pd.read_csv('end_res.csv')


In [5]:
def sim_gen(abs):
    sys_prompt = "Given a medical abstract, generate a simplified version that anyone can understand."
    
    response = openai.ChatCompletion.create(
        model='gpt-4', 
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": "Medical abstract:\n\n{}\n".format(abs)}
        ]
    )
    
    return response['choices'][0]['message']['content']

In [6]:
df['simplification'] = df.apply(lambda x: sim_gen(x.Abstract), axis=1)

In [7]:
df.to_csv('end_res_sim.csv')

In [8]:
df[['Abstract', 'simplification']]

,Abstract,simplification
0,ABSTRACT.BACKGROUND: Antibiotics are overused ...,Simple Explanation:\n\nAntibiotics are sometim...
1,ABSTRACT.BACKGROUND: Minimal hepatic encephalo...,Simplified version:\n\nThis study was about a ...
2,ABSTRACT.OBJECTIVE: To determine whether glarg...,Simple explanation:\n\nThe aim of this study w...
3,ABSTRACT.BACKGROUND: ACE inhibition results in...,Simplified version:\n\nThis study was about a ...
4,ABSTRACT.BACKGROUND:: Oral isotretinoin is hig...,Simplified version:\n\nThe study focuses on th...


In [15]:
df['Abstract'][0][df['Evidence Start'][0].item():df['Evidence End'][0].item()]

're was reduced from 6.3 to 4.5 days under PCT guidance (−1.8 days; 95% CI −3.1, −0.5; P = 0.039) for all LRTI and from 9.1 to 5.7 days '

In [16]:
df['Annotations'][0]

'Antibiotic prescribing rates were not significantly different in PCT guided patients compared to controls (OR 1.26; 95% CI 0.81, 1.95).'

In [17]:
df.head()

,Unnamed: 0,UserID,PromptID,PMCID,Valid Label,Valid Reasoning,Label,Annotations,Label Code,In Abstract,Evidence Start,Evidence End,Abstract,to_drop,simplification
0,1,0,2193,3735552,True,True,no significant difference,Antibiotic prescribing rates were not signific...,0,True,1076,1211,ABSTRACT.BACKGROUND: Antibiotics are overused ...,True,Simple Explanation:\n\nAntibiotics are sometim...
1,3,0,6921,5636096,True,True,significantly increased,The patients with an abnormal CRT index who we...,1,True,1551,1720,ABSTRACT.BACKGROUND: Minimal hepatic encephalo...,True,Simplified version:\n\nThis study was about a ...
2,7,0,78,2875419,True,True,no significant difference,Improvements in A1C were −1.46 ± 1.09% for gla...,0,True,834,927,ABSTRACT.OBJECTIVE: To determine whether glarg...,True,Simple explanation:\n\nThe aim of this study w...
3,34,0,10783,2039813,True,True,no significant difference,The rate of change in FMD per 6 months was 0.1...,0,True,1281,1458,ABSTRACT.BACKGROUND: ACE inhibition results in...,True,Simplified version:\n\nThis study was about a ...
4,36,1,11954,5038098,True,True,no significant difference,"At the end of therapy, decrease in the total a...",0,True,780,1016,ABSTRACT.BACKGROUND:: Oral isotretinoin is hig...,True,Simplified version:\n\nThe study focuses on th...


In [21]:
new_data = []
for ind, row in df.iterrows():
    entry = {
        "source": row['Abstract'].replace("<", "&lt;").replace(">", "&gt;"),
        "target": row['simplification'].replace("<", "&lt;").replace(">", "&gt;"),
        "edits": [
            {
                "category": "hol_eval",
                "id": 0
            },
            {
                "category": "final_q",
                "id": 1
            },
            {
                "category": "pico_pop",
                "id": 2
            },
            {
                "category": "pico_inter",
                "id": 3
            },
            {
                "category": "pico_comp",
                "id": 4
            },
            {
                "category": "pico_out",
                "id": 5
            },
            {
                "category": "results",
                "id": 6,
                "input_idx": [
                    [
                        row['Abstract'].replace("<", "&lt;").replace(">", "&gt;").find(row['Annotations'].replace("<", "&lt;").replace(">", "&gt;")),
                        row['Abstract'].replace("<", "&lt;").replace(">", "&gt;").find(row['Annotations'].replace("<", "&lt;").replace(">", "&gt;"))+len(row["Annotations"].replace("<", "&lt;").replace(">", "&gt;"))
                    ]
                ]
            }
        ]   
    }
    new_data.append(entry)

In [22]:
with open('results_test.json', 'w') as d:
    json.dump(new_data, d, indent=4)